## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [22]:
ib.disconnect()

In [25]:
from ib_insync import *
import pandas as pd
util.startLoop()
from datetime import date
from datetime import datetime

# ----------------- DEFINE STRATEGY LIMITS ------------------
days_mins_A, days_mins_B = 34, 57
days_maxs_A, days_maxs_B = 90, 150
delta_mins_A, delta_mins_B = 0.1, 0.2
delta_maxs_A, delta_maxs_B = 0.2, 0.3


# ----------------- DATE HELPING FUNCTIONS ------------------
def verify_daycount(dates):
    ds = []
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    for adate in dates:
        date_obj = datetime.strptime(adate, "%Y%m%d")
        days_diff = (date_obj - today).days
        if (days_mins_A < days_diff < days_mins_B) or (days_maxs_A < days_diff < days_maxs_B):
            ds.append(adate)
    return ds

def days_diff(adate):
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    date_obj = datetime.strptime(adate, "%Y%m%d")
    return (date_obj - today).days
    

# ----------------- IB.API SETTINGS ------------------ v
ib = IB()
ib.disconnect()
ib.connect('127.0.0.1', 7497, clientId = 2)

tick = 'AAPL'
stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) 
chains = pd.DataFrame(ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId))
expirations = sorted(chains['expirations'].iloc[0])
expirations = verify_daycount(expirations) 
strikes = chains['strikes'].iloc[0]
contractID = chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] 


# -------------------- IB.API CONNECTION ------------------------------
ds = pd.DataFrame()

contracts = [Option(tick, ex, s, r, 'SMART') for r in rights for ex in expirations for s in strikes]
qualifiedContracts = ib.qualifyContracts(*contracts)
qualifiedContractDetails = ib.reqTickers(*qualifiedContracts)
for qcd in qualifiedContractDetails:
    try:
        details = qcd.contract
        exp, strike, right = details.lastTradeDateOrContractMonth, details.strike, details.right
        greek = qcd.modelGreeks
        delta, theta, gamma, vega, pvDiv, optPrice, impliedVol = greek.delta, greek.theta, greek.gamma, greek.vega, greek.pvDividend, greek.optPrice, greek.impliedVol
        row = {
            'Symbol': tick, 'Expiration': exp, 'Strike': strike, 'Right': right,
            'Delta': delta, 'Theta': theta, 'Vega': vega, 'Gamma': gamma,
            'impVol': impliedVol, 'optPrice': optPrice, 'Ask': qcd.ask, 'Bid': qcd.bid, 'Last': qcd.last}
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)
    except:
        pass


# -------------------- CREATE BUNDLES ------------------------------
ds['daysdiff'] = list(map(days_diff, ds['Expiration'])) 
ds['Delta'], ds['Theta'] = ds['Delta'].apply(lambda x: round(abs(x), 2)), ds['Theta'].apply(lambda x: round(abs(x), 2))

ds_p1 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= days_maxs_A) & (ds['daysdiff'] <= days_maxs_B) & (ds['Delta'] >= delta_mins_A) & (ds['Delta'] <= delta_mins_B)]
ds_p1['REGEL'] = ['P1'] * len(ds_p1)
ds_p2 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= days_mins_A) & (ds['daysdiff'] <= days_mins_B) & (ds['Delta'] >= delta_maxs_A) & (ds['Delta'] <= delta_maxs_B)]
ds_p2['REGEL'] = ['P2'] * len(ds_p2)
ds_c3 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= days_mins_A) & (ds['daysdiff'] <= days_mins_B) & (ds['Delta'] >= delta_maxs_A) & (ds['Delta'] <= delta_maxs_B)]
ds_c3['REGEL'] = ['C3'] * len(ds_c3)
ds_c4 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= days_maxs_A) & (ds['daysdiff'] <= days_maxs_B) & (ds['Delta'] >= delta_mins_A) & (ds['Delta'] <= delta_mins_B)]
ds_c4['REGEL'] = ['C4'] * len(ds_c4)


main = pd.concat([ds_p1, ds_p2, ds_c3, ds_c4], ignore_index = True)
def marg_condition(a, b): # (p1, p2) (c3, c4)
    if b - 20 <= a <= b - 10: return True
    else: return False

def theta_condition(a, b): #(p2, p1) (c3, c4)
    if a > b * 1.5: return True
    else: return False

def bundle(ds):
    p1 = ds[ds['REGEL'] == 'P1']
    p2 = ds[ds['REGEL'] == 'P2']
    c3 = ds[ds['REGEL'] == 'C3']
    c4 = ds[ds['REGEL'] == 'C4']
    ps, cs, i = [], [], 0

    for i in range(len(p1)):
        for j in range(len(p2)):
            if marg_condition(p1.iloc[i]['Strike'], p2.iloc[j]['Strike']) & theta_condition(p2.iloc[j]['Theta'], p1.iloc[i]['Theta']):
                ps.append((i, j))

    for i in range(len(c3)):
        for j in range(len(c4)):
            if marg_condition(c3.iloc[i]['Strike'], c4.iloc[j]['Strike']) & theta_condition(c3.iloc[i]['Theta'], c4.iloc[j]['Theta']):
                cs.append((i, j))
    
    legs_indx, dss = [(p1, p2, c3, c4) for (p1, p2) in ps for (c3, c4) in cs], []
    for (a, b, c, d) in legs_indx:
        dss.append(pd.concat([p1.iloc[a].to_frame().T, p2.iloc[b].to_frame().T, c3.iloc[c].to_frame().T, c4.iloc[d].to_frame().T], ignore_index=True))
    
    return dss

bundles = bundle(main)
pd.concat(bundles).to_csv(tick+'_options.csv')


# -------------------- CREATE BUNDLE SUMMARY LINE ------------------------------
contract = Stock(tick,'SMART','USD')
m_data = ib.reqMktData(contract)
while m_data.last != m_data.last: ib.sleep(0.01) 
last_price = m_data.last

ticker = ib.reqMktData(contract, '165')
while ticker.low26week != ticker.low26week: ib.sleep(0.01)
low26w, high26w, change = ticker.low26week, ticker.high26week, ((ticker.close - last_price) / last_price) * 100


d = " - "
mids = { #main info dataset
    'Symbol': tick, 
    'Last': last_price,
    'Change%': round(change, 2),
    'Low-26W': round(low26w, 2),
    'High-26W': round(high26w, 2),
    "IV%": d,
    "H-IV-30%": d,
    "H-IV-Mean%": 23,
    "RSI-14D": d
}

mids = pd.DataFrame([mids])
print(mids)



ib.disconnect()

Error 200, reqId 5: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221111', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221111', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221111', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221111', strike=45.0, right='P', exchange='SMART')
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20221111', strike=50.0, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has be

  Symbol    Last  Change%  Low-26W  High-26W  IV% H-IV-30%  H-IV-Mean% RSI-14D
0   AAPL  140.22     3.72   128.86    176.15   -        -           23      - 


In [26]:
print(ds)

    Symbol Expiration  Strike Right  Delta  Theta      Vega     Gamma  \
0     AAPL   20221111    70.0     P   0.00   0.01  0.005008  0.000266   
1     AAPL   20221111    75.0     P   0.01   0.01  0.007063  0.000403   
2     AAPL   20221111    80.0     P   0.01   0.01  0.008752  0.000608   
3     AAPL   20221111    85.0     P   0.01   0.01  0.014123  0.000881   
4     AAPL   20221111    90.0     P   0.02   0.02  0.016343  0.001278   
..     ...        ...     ...   ...    ...    ...       ...       ...   
519   AAPL   20230217   280.0     C   0.00   0.00  0.005698  0.000186   
520   AAPL   20230217   285.0     C   0.00   0.00  0.004352  0.000159   
521   AAPL   20230217   290.0     C   0.00   0.00  0.004372  0.000153   
522   AAPL   20230217   295.0     C   0.00   0.00  0.004387  0.000153   
523   AAPL   20230217   300.0     C   0.00   0.00  0.004406  0.000150   

       impVol  optPrice   Ask   Bid  Last  daysdiff  
0    0.876552  0.042818  0.05  0.04  0.04        35  
1    0.833634  

In [51]:
import openpyxl
import xlsxwriter
import numpy as np
import math

# ------------------------- HELPER FUNCTIONS TO DECIDE GRADES --------------------------
def calculate_grade(value, graph_points): # TODO: Work on algoirthm
        keys, values = list(graph_points.keys()), list(graph_points.values())
        if (value < keys[0]): return values[0]
        elif (value > keys[len(graph_points) - 1]): return values[len(graph_points) - 1]
        else:
                i = 0
                for i in range(len(keys) - 1):
                        if (keys[i] <= value) & (value <= keys[i+1]):
                                x2, x1 = keys[i + 1] - keys[i], 0
                                y2, y1 = values[i + 1], values[i]
                                b = values[i]
                                m = (y2 - y1)/(x2 - x1)
                                return  round(m * (value - keys[i]) + b, 2) # y = m * x + b
                        i += 1
        return -1

def calculate_delta_ratio_grade(delta_p1, delta_p2, delta_c3, delta_c4):
        p2c3 = {0.20:65 , 0.21:70 , 0.22:75 , 0.23:85 , 0.24:90 , 0.25:100 , 0.26:90 , 0.27:85 , 0.28:75 , 0.29:70 , 0.30:65}
        p1c4 = {0.1:60, 0.11:65 , 0.12:70 , 0.13:75 , 0.14:85 , 0.15:90 , 0.16:100 , 0.17:90 , 0.18:85 , 0.19:75 , 0.20:70 , 0.21:65}
        ratio = {0.59:65 , 0.60:70 , 0.61:75 , 0.62:85 , 0.63:90 , 0.64:100 , 0.65:90 , 0.66:85 , 0.67:75 , 0.68:70 , 0.69:65}
        
        singles = p1c4[delta_p1] * 0.2 + p2c3[delta_p2] * 0.3 + p2c3[delta_c3] * 0.3 + p1c4[delta_c4] * 0.2
        pratio, cratio = round(delta_p1/delta_p2, 2), round(delta_c4/delta_c3, 2)
        pratio_score, cratio_score = 0.0, 0.0

        if pratio in list(ratio.keys()):
                pratio_score = ratio[pratio] * 0.5
        elif (pratio >= 0.5 and pratio < 0.59 ) or (pratio > 0.69 and pratio <= 74):
                pratio_score = 50 * 0.5
        else: 
                pratio_score = 0

        if cratio in list(ratio.keys()):
                cratio_score = ratio[cratio] * 0.5
        elif (cratio >= 0.5 and cratio < 0.59 ) or (cratio > 0.69 and cratio <= 74):
                cratio_score = 50 * 0.5
        else: 
                cratio_score = 0

        return singles * 0.3 + (pratio_score + cratio_score) * 0.7

# bundles.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

# writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
# workbook = writer.book
# worksheet = workbook.add_worksheet('Result')
# writer.sheets['Bundles'] = worksheet

# mids.to_excel(writer, sheet_name = 'Bundles', startrow = 0, startcol = 0)


bundle_count = 1
currow, curcol = 4, 0
packages = []
max_yearly, max_rtm_p, max_rtm_c, max_stktop1,  max_stktop2,  max_stktoc3, max_stktoc4 = 0, 0, 0, 0, 0, 0, 0
min_ivpc = 100000 #Change later for convenience reasons
for ds in bundles:

        package = []

        # ----------------- OPTION DF -------------------------
        symbol = str(list(ds[ds['REGEL'] == 'P1']['Symbol'])[0])
        # worksheet.write_string(currow, 0, str(symbol) + " " + str(bundle_count) + "/" + str(len(bundles)))
        ds['Change'] = ((ds['Strike'] - last_price)/last_price) #convert to percentage
        ds['Change'] = ds['Change'].apply(lambda x: round(x, 2))
        ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
        ds.loc[ds.REGEL == 'P2', 'Mark'] *= -1
        ds.loc[ds.REGEL == 'C3', 'Mark'] *= -1
        ds['Mark'] = ds['Mark'].apply(lambda x: round(x, 2))
        ds = ds.rename({'impVol': 'IV-OP'}, axis = 1)
        ds['IV-OP'] = ds['IV-OP'].apply(lambda x: round(x, 2))
        # ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds['Expiration'] = pd.to_datetime(ds.Expiration, format='%Y%m%d')
        ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds = ds.rename({'daysdiff': 'Days'}, axis = 1) 
        ds['Delta'] = ds['Delta'].apply(lambda x: round(x, 2))
        ds['Theta'] = ds['Theta'].apply(lambda x: round(x, 2))
        ds['Vega'] = ds['Vega'].apply(lambda x: round(x, 2))


        mp, mc = list(ds[ds['REGEL'] == 'P2']['Strike'])[0] - list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0] - list(ds[ds['REGEL'] == 'C3']['Strike'])[0]
        vega_p1, vega_p2, vega_c3, vega_c4 = list(ds[ds['REGEL'] == 'P1']['Vega'])[0], list(ds[ds['REGEL'] == 'P2']['Vega'])[0], list(ds[ds['REGEL'] == 'C3']['Vega'])[0], list(ds[ds['REGEL'] == 'C4']['Vega'])[0]
        delta_p1, delta_p2, delta_c3, delta_c4 = list(ds[ds['REGEL'] == 'P1']['Delta'])[0], list(ds[ds['REGEL'] == 'P2']['Delta'])[0], list(ds[ds['REGEL'] == 'C3']['Delta'])[0], list(ds[ds['REGEL'] == 'C4']['Delta'])[0]
        theta_p1, theta_p2, theta_c3, theta_c4 = list(ds[ds['REGEL'] == 'P1']['Theta'])[0], list(ds[ds['REGEL'] == 'P2']['Theta'])[0], list(ds[ds['REGEL'] == 'C3']['Theta'])[0], list(ds[ds['REGEL'] == 'C4']['Theta'])[0]
        days_p1, days_p2, days_c3, days_c4 = list(ds[ds['REGEL'] == 'P1']['Days'])[0], list(ds[ds['REGEL'] == 'P2']['Days'])[0], list(ds[ds['REGEL'] == 'C3']['Days'])[0], list(ds[ds['REGEL'] == 'C4']['Days'])[0]
        iv_p1, iv_p2, iv_c3, iv_c4 = list(ds[ds['REGEL'] == 'P1']['IV-OP'])[0], list(ds[ds['REGEL'] == 'P2']['IV-OP'])[0], list(ds[ds['REGEL'] == 'C3']['IV-OP'])[0], list(ds[ds['REGEL'] == 'C4']['IV-OP'])[0]
        mark_p1, mark_p2, mark_c3, mark_c4 = list(ds[ds['REGEL'] == 'P1']['Mark'])[0], list(ds[ds['REGEL'] == 'P2']['Mark'])[0], list(ds[ds['REGEL'] == 'C3']['Mark'])[0], list(ds[ds['REGEL'] == 'C4']['Mark'])[0]
        strike_p1, strike_p2, strike_c3, strike_c4 = list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'P2']['Strike'])[0], list(ds[ds['REGEL'] == 'C3']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0]


        rtd_p, rtd_c = list(ds[ds['REGEL'] == 'P2']['Theta'])[0] - list(ds[ds['REGEL'] == 'P1']['Theta'])[0], list(ds[ds['REGEL'] == 'C3']['Theta'])[0] - list(ds[ds['REGEL'] == 'C4']['Theta'])[0]
        rtm_p, rtm_c = rtd_p * 22, rtd_c * 22
        rty_p, rty_c = rtm_p * 12, rtm_c * 12
        ds['RTD'] = [rtd_p, rtd_p, rtd_c, rtd_c]
        ds['RTM'] = [rtm_p, rtm_p, rtm_c, rtm_c]
        ds['RTY'] = [rty_p, rty_p, rty_c, rty_c]
        ds['Margin'] = [mp, mp, mc, mc]


        title = str(symbol) + " " + str(bundle_count) + "/" + str(len(bundles))
        ds = ds.rename({'REGEL': title}, axis = 1)
        ds.set_index(title, inplace = True)
        ds_bundle = ds[['Expiration', 'Days', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'IV-OP', 'RTM']]
        package.append((ds_bundle, currow))
        # ds[['Expiration', 'Days', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'IV-OP', 'RTM']].to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += ds.shape[0] + 2

        # ----------------- NATIVE DF -------------------------
        mc = mc.astype(np.int64)
        mp = mp.astype(np.int64)
        
        if mc > mp: max_margin = mc
        else: max_margin = mp

        pnative = {
        'Type': "P-Native", 
        'Margin': mp,
        'Monthly%': round((rtd_p * 22 * 100) / mp, 2),
        'Yearly%': round((rtd_p * 22 * 12 * 100) / mp, 2),
        'Yearly$': rtd_p * 22 * 12,
        "Cost": list(ds[ds.index == 'P1']['Mark'])[0] + list(ds[ds.index == 'P2']['Mark'])[0]
        }

        cnative = {
        'Type': "C-Native", 
        'Margin': mc,
        'Monthly%': round((rtd_c * 22 * 100) / mc, 2),
        'Yearly%': round((rtd_c * 22 * 12 * 100) / mc, 2),
        'Yearly$': (rtd_c * 22 * 12),
        "Cost": list(ds[ds.index == 'C3']['Mark'])[0] + list(ds[ds.index == 'C4']['Mark'])[0]
        }

        posnative = {
        'Type': "POS-Native", 
        'Margin': max_margin,
        'Monthly%': round(((rtd_c * 22 + rtd_p * 22) / max_margin) * 100, 2),
        'Yearly%': round((((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max_margin) * 100, 2),
        'Yearly$': (pnative['Yearly$'] + cnative['Yearly$']),
        "Cost": ds['Mark'].sum()
        }

        ds_native = pd.DataFrame([pnative, cnative, posnative])
        ds_native.set_index('Type', inplace = True)
        # ds_native.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        package.append((ds_native, currow))
        currow += 5 # 4 for the native ds_, 2 for spacing

        # ----------------- GRADES DF -------------------------
        
                

        #0 = TODO
        #-1 = range not ok
        #-2 = to implement after code structural review

        # ----------------- OPTIMUM DF -------------------------
        yearly = ((rty_c + rty_p) / max_margin) * 100
        optimum_value = {
        'OPTIMUM': "Value", 
        'Total': " - ", # TODO 
        'Yearly%': yearly,
        'Delta Ratio': " - ", # NO VALUE HERE 
        'Delta Neutral': delta_p1 + delta_p2 + delta_c3 + delta_c4,
        'Vega Neutral': (vega_p2 + vega_c3) - (vega_p1 + vega_c4),
        'Day P1': days_p1,
        'Day P2': days_p2,
        'Day C3': days_c3,
        'Day C4': days_c4,
        'Stk Dist Mid 26W': round((low26w + high26w) / 2, 2),
        'IV Gap Stk': 0, # Distance between Stock_IV to 23 (Mean IV)
        'IV P/C': round(iv_p2 / iv_c3, 2),
        'P Margin': mp,
        'C Margin': mc,
        'Max Margin': max_margin,
        'RSI': last_price
        }

        optimum_grade = {
        'OPTIMUM': 'Grade',
        'Total': ' - ',
        'Yearly%': -2,
        'Delta Ratio': calculate_delta_ratio_grade(delta_p1, delta_p2, delta_c3, delta_c4),
        'Delta Neutral': calculate_grade(optimum_value['Delta Neutral'], {-0.15:10, 0:100 , 0.15:10}),
        'Vega Neutral': calculate_grade(optimum_value['Vega Neutral'], {-0.15:10, 0:100 , 0.15:10 }),
        'Day P1': calculate_grade(optimum_value['Day P1'], {60: 0, 128: 100}),
        'Day P2': calculate_grade(optimum_value['Day P2'], {34:10, 45:100, 57:10}),
        'Day C3': calculate_grade(optimum_value['Day C3'], {34:10, 45:100, 57:10}),
        'Day C4': calculate_grade(optimum_value['Day C4'], {60: 0, 128: 100}),
        'Stk Dist Mid 26W': calculate_grade(optimum_value['Stk Dist Mid 26W'], {(last_price - 15.0):10, last_price:100, (last_price + 30.0):10}),
        'IV Gap Stk': calculate_grade(optimum_value['IV Gap Stk'], {-3:0, -3:30 , 0:30 , 0:20 , 2:20 , 7:100 , 7:15}),
        'IV P/C': calculate_grade(optimum_value['IV P/C'], {0.8:0, 1.1:100, 1.4:0}),
        'P Margin': calculate_grade(optimum_value['P Margin'], {5:100 , 10:60 , 15:0}),
        'C Margin': calculate_grade(optimum_value['C Margin'], {10:100 , 15:75 , 20:0}),
        'Max Margin': calculate_grade(optimum_value['Max Margin'], {10:100 , 15:75 , 20:0}),
        'RSI': calculate_grade(optimum_value['RSI'], {30:0,  40:100 , 50:100 , 60:0})
        }

        optimum_weight = {
        'OPTIMUM': "Weight", 
        'Total': " - ",
        'Yearly%': optimum_grade['Yearly%'] * 0.20,
        'Delta Ratio': optimum_grade['Delta Ratio'] * 0.10,
        'Delta Neutral': optimum_grade['Delta Neutral'] * 0.05,
        'Vega Neutral': optimum_grade['Vega Neutral'] * 0.05,
        'Day P1': optimum_grade['Day P1'] * 0.05,
        'Day P2': optimum_grade['Day P2'] * 0.05,
        'Day C3': optimum_grade['Day C3'] * 0.05,
        'Day C4': optimum_grade['Day C4'] * 0.05,
        'Stk Dist Mid 26W': optimum_grade['Stk Dist Mid 26W'] * 0.05,
        'IV Gap Stk': optimum_grade['IV Gap Stk'] * 0.05,
        'IV P/C': optimum_grade['IV P/C'] * 0.05,
        'P Margin': optimum_grade['P Margin'] * 0.05,
        'C Margin': optimum_grade['C Margin'] * 0.05,
        'Max Margin': optimum_grade['Max Margin'] * 0.10,
        'RSI': optimum_grade['RSI'] * 0.10
        }

        ds_optimum = pd.DataFrame([optimum_value, optimum_grade, optimum_weight])
        ds_optimum.set_index('OPTIMUM', inplace = True)
        # ds_optimum.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        package.append((ds_optimum, currow))

        currow += 5 # 4 for the ds_
        # ----------------- PROFIT DF -------------------------
        profit_value = {
        'PROFIT': "Value", 
        'Total': " - ", # TODO 
        'Cost-P': round((mark_p1 + mark_p2) / mark_p1, 2), 
        'Cost-C': round((mark_c3 + mark_c4) / mark_c4, 2), 
        'Cost-P/C': round((mark_p1 + mark_c4) / (mark_p1 + mark_p2 + mark_c3 + mark_c4), 2),
        'Theta-P1/P2': round(theta_c3/theta_c4 ,2),
        'Theta-C3/C4': round(theta_p2/theta_p1 ,2),
        'RTM-P': round(rtm_p, 2),
        'RTM-C': round(rtm_c, 2),
        'IV-P-Gap': round(iv_p2 - iv_p1 ,2),
        'IV-C-Gap': round(iv_c3 - iv_c4 ,2)
        }

        profit_grade = {
        'PROFIT': "Grade", 
        'Total': " - ", # TODO 
        'Cost-P': calculate_grade(profit_value['Cost-P'], {0:100 , 0.1:100, 0.5:0}),
        'Cost-C': calculate_grade(profit_value['Cost-P'], {0:100 , 0.1:100, 0.5:0}),
        'Cost-P/C': calculate_grade(profit_value['Cost-P/C'], {0:100 , 0.1:100, 0.5:0}),
        'Theta-P1/P2': calculate_grade(profit_value['Theta-P1/P2'], {0:20 , 1.5:20, 2.1:100}),
        'Theta-C3/C4': calculate_grade(profit_value['Theta-C3/C4'], {0:20 , 1.5:20, 2.1:100}),
        'RTM-P': -2,
        'RTM-C': -2,
        'IV-P-Gap': calculate_grade(profit_value['IV-P-Gap'], {0.5:0, 2.0:30, 4.0:70, 8.0:100}),
        'IV-C-Gap': calculate_grade(profit_value['IV-C-Gap'], {0.5:0, 2.0:30, 4.0:70, 8.0:100})
        }

        profit_weight = {
        'PROFIT': "Weight", 
        'Total': " - ", # TODO 
        'Cost-P': profit_grade['Cost-P'] * 0.10,
        'Cost-C': profit_grade['Cost-C'] * 0.10,
        'Cost-P/C': profit_grade['Cost-P/C'] * 0.10,
        'Theta-P1/P2': profit_grade['Theta-P1/P2'] * 0.10,
        'Theta-C3/C4': profit_grade['Theta-C3/C4'] * 0.10,
        'RTM-P': profit_grade['RTM-P'] * 0.05,
        'RTM-C': profit_grade['RTM-C'] * 0.05,
        'IV-P-Gap': profit_grade['IV-P-Gap'] * 0.10,
        'IV-C-Gap':profit_grade['IV-P-Gap'] * 0.10
        }            

        ds_profit = pd.DataFrame([profit_value, profit_grade, profit_weight])
        ds_profit.set_index('PROFIT', inplace = True)
        # ds_profit.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        package.append((ds_profit, currow))
        currow += 5 # 4 for the ds_


        # ----------------- RISK DF -------------------------
        stktop1, stktop2, stktoc3, stktoc4 = (((last_price - strike_p1) / days_p1) / last_price) * 100, (((last_price - strike_p2) / days_p2) / last_price) * 100, (((last_price - strike_c3) / days_c3) / last_price) * 100, (((last_price - strike_c4) / days_c4) / last_price) * 100
        current_ivpc = round(iv_p2 / iv_c3 ,2)
        risk_value = {
        'RISK': "Value", 
        'Total': " - ", 
        'VIX': -2, 
        'DELTA-RISK-P1': delta_p1, 
        'DELTA-RISK-P2': delta_p2,
        'DELTA-RISK-P2/C3': round((1 - delta_p2) * (1 - delta_c3), 2),
        'DELTA-RISK-P1/C4': round((1 - delta_p1) * (1 - delta_c4), 2),
        'DELTA-RISK-C3': delta_c3,
        'DELTA-RISK-C4': delta_c4,
        'STK-TO-P1': round(stktop1, 2),
        'STK-TO-P2': round(stktop2, 2),
        'STK-TO-C3': round(stktoc3, 2),
        'STK-TO-C4': round(stktoc4, 2),
        'IVP/IVC': current_ivpc,
        'C3-HIGH-26W': high26w - strike_c3,
        'P2-LOW-26W': low26w - strike_p2
        }

        risk_grade = {
        'RISK': "Grade", 
        'Total': -2, 
        'VIX': calculate_grade(risk_value['VIX'], {15:100 , 20:10}),
        'DELTA-RISK-P1': calculate_grade(risk_value['DELTA-RISK-P1'], {0.10:100, 0.25:0}),
        'DELTA-RISK-P2': calculate_grade(risk_value['DELTA-RISK-P2'], {0.20:100, 0.30:0}), 
        'DELTA-RISK-P2/C3': calculate_grade(risk_value['DELTA-RISK-P2/C3'], {0.49:0, 0.55:10 , 0.64:100}), 
        'DELTA-RISK-P1/C4': calculate_grade(risk_value['DELTA-RISK-P1/C4'], {0.60:50, 0.81:100}),
        'DELTA-RISK-C3': calculate_grade(risk_value['DELTA-RISK-C3'], {0.20:100, 0.30:0}), 
        'DELTA-RISK-C4': calculate_grade(risk_value['DELTA-RISK-C4'], {0.10:100, 0.20:0}), 
        'STK-TO-P1': -2,
        'STK-TO-P2': -2,
        'STK-TO-C3': -2,
        'STK-TO-C4': -2,
        'IVP/IVC': -2,
        'C3-HIGH-26W': calculate_grade(risk_value['C3-HIGH-26W'], {-10:30 , 0:100 , 10:30}),
        'P2-LOW-26W': calculate_grade(risk_value['P2-LOW-26W'], {-10:30 , 0:100 , 10:30})
        }

        risk_weight = {
        'RISK': "Weight", 
        'Total': -2,  
        'VIX': risk_grade['VIX'] * 0.15,
        'DELTA-RISK-P1': risk_grade['DELTA-RISK-P1'] * 0.05,
        'DELTA-RISK-P2': risk_grade['DELTA-RISK-P2'] * 0.05,
        'DELTA-RISK-P2/C3': risk_grade['DELTA-RISK-P2/C3'] * 0.10,
        'DELTA-RISK-P1/C4': risk_grade['DELTA-RISK-P1/C4'] * 0.05,
        'DELTA-RISK-C3': risk_grade['DELTA-RISK-C3'] * 0.05,
        'DELTA-RISK-C4': risk_grade['DELTA-RISK-C4'] * 0.05,
        'STK-TO-P1': 0,
        'STK-TO-P2': 0,
        'STK-TO-C3': 0,
        'STK-TO-C4': 0,
        'IVP/IVC': 0,
        'C3-HIGH-26W': risk_grade['C3-HIGH-26W'] * 0.10,
        'P2-LOW-26W': risk_grade['P2-LOW-26W'] * 0.10
        }            

        ds_risk = pd.DataFrame([risk_value, risk_grade, risk_weight])
        ds_risk.set_index('RISK', inplace = True)
        # ds_risk.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        package.append((ds_risk, currow))
        currow += 7 # 4 for the ds_, 3 for spacing
        packages.insert(bundle_count - 1, package)

        # ------------------ CALCULATE MAXS ---------------------
        max_yearly = max(max_yearly, yearly)
        max_rtm_c = max(max_rtm_c, rtm_c)
        max_rtm_p = max(max_rtm_p, rtm_p)
        max_stktop1 = max(max_stktop1, abs(stktop1))
        max_stktop2 = max(max_stktop2, abs(stktop2))
        max_stktoc3 = max(max_stktoc3, abs(stktoc3))
        max_stktoc4 = max(max_stktoc4, abs(stktoc4))
        min_ivpc = min(min_ivpc, current_ivpc)

        bundle_count += 1

# writer.save() #save the excelt sheet

In [52]:
#calculating missing values based on categorical maxes

for package in packages:
    (ds, currow) = package[2]
    ds.at['Grade', 'Yearly%'] = (ds.at['Value', 'Yearly%'] * 100 / max_yearly) - 10
    ds.at['Weight', 'Yearly%'] = ds.at['Grade', 'Yearly%'] * 0.20
    package[2] = (ds, currow)

        

    (ds, currow) = package[3] # profit
    ds.at['Grade', 'RTM-P'] = ds.at['Value', 'RTM-P'] * 100 / max_rtm_p
    ds.at['Weight', 'RTM-P'] = ds.at['Grade', 'RTM-P'] * 0.05
    ds.at['Grade', 'RTM-C'] = ds.at['Value', 'RTM-C'] * 100 / max_rtm_c
    ds.at['Weight', 'RTM-C'] = ds.at['Grade', 'RTM-C'] * 0.05
    package[3] = (ds, currow)



    (ds, currow) = package[4] #risk
    ds.at['Grade', 'STK-TO-P1'] = ds.at['Value', 'STK-TO-P1'] * 100 / max_stktop1
    ds.at['Weight', 'STK-TO-P1'] = ds.at['Grade', 'STK-TO-P1'] * 0.05
    ds.at['Grade', 'STK-TO-P2'] = ds.at['Value', 'STK-TO-P2'] * 100 / max_stktop2
    ds.at['Weight', 'STK-TO-P2'] = ds.at['Grade', 'STK-TO-P2'] * 0.05
    ds.at['Grade', 'STK-TO-C3'] = ds.at['Value', 'STK-TO-C3'] * 100 / max_stktoc3
    ds.at['Weight', 'STK-TO-C3'] = ds.at['Grade', 'STK-TO-C3'] * 0.05
    ds.at['Grade', 'STK-TO-C4'] = ds.at['Value', 'STK-TO-C4'] * 100 / max_stktoc4
    ds.at['Weight', 'STK-TO-C4'] = ds.at['Grade', 'STK-TO-C4'] * 0.05
    ds.at['Grade', 'IVP/IVC'] = ds.at['Value', 'IVP/IVC'] * 100 / min_ivpc
    ds.at['Weight', 'IVP/IVC'] = ds.at['Grade', 'IVP/IVC'] * 0.05
    package[4] = (ds, currow) 
    



In [53]:
#print all 

writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Bundles')
writer.sheets['Bundles'] = worksheet

yellow = workbook.add_format({'bg_color': 'yellow'})
green = workbook.add_format({'bg_color': 'green'})
red = workbook.add_format({'bg_color': 'red'})

mids.to_excel(writer, sheet_name = 'Bundles', startrow = 0, startcol = 0)

for package in packages:
    for (printage, row) in package:
        printage.to_excel(writer, sheet_name = 'Bundles', startrow = row, startcol = 0)

writer.save() #save the excelt sheet